# High-level RNN Keras (TF) Example

*Modified by Jordan A Caraballo Vega (jordancaraballo)*

In [1]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "tensorflow"

import warnings # make notebook more readable and nice
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import keras as K
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, CuDNNGRU
from common.params_lstm import *
from common.utils import *

Using TensorFlow backend.


In [2]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tf.__version__)
print(K.backend.backend())
print(K.backend.image_data_format())
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.6.7 | packaged by conda-forge | (default, Jul  2 2019, 02:18:42) 
[GCC 7.3.0]
Keras:  2.2.4
Numpy:  1.17.0
Tensorflow:  1.13.1
tensorflow
channels_last
GPU:  ['Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB']
CUDA Version 10.1.243
CuDNN Version  No CUDNN in this machine


In [4]:
def create_symbol(CUDNN=True, maxf=MAXFEATURES, edim=EMBEDSIZE, nhid=NUMHIDDEN, maxl=MAXLEN):
    model = Sequential()
    model.add(Embedding(maxf, edim, input_length=maxl))
    # Only return last output
    if not CUDNN:
        model.add(GRU(nhid, return_sequences=False, return_state=False))
    else:
        model.add(CuDNNGRU(nhid, return_sequences=False, return_state=False))
    model.add(Dense(2, activation='softmax'))
    return model

In [5]:
def init_model(m, lr=LR, b1=BETA_1, b2=BETA_2, eps=EPS):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.Adam(lr, b1, b2, eps),
        metrics = ['accuracy'])
    return m

In [6]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = imdb_for_library(seq_len=MAXLEN, max_features=MAXFEATURES, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
Trimming to 30000 max-features
Padding to length 150


/home/jacaraba/.conda/envs/ngaml/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:441: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. The passed value of 'all' is the default and can simply be removed.
  DeprecationWarning)


(25000, 150) (25000, 150) (25000, 2) (25000, 2)
int32 int32 int32 int32
CPU times: user 4.91 s, sys: 425 ms, total: 5.34 s
Wall time: 5.42 s


In [7]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 3.42 s, sys: 9.42 s, total: 12.8 s
Wall time: 13.8 s


In [8]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 26.5 ms, sys: 322 µs, total: 26.9 ms
Wall time: 25.9 ms


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 125)          3750000   
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (None, 100)               68100     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 3,818,302
Trainable params: 3,818,302
Non-trainable params: 0
_________________________________________________________________


In [10]:
%%time
# Main training loop: 26s
EPOCHS=40
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

Epoch 1/40
25000/25000 [==============================] - 49s 2ms/step - loss: 0.5173 - acc: 0.7168
Epoch 2/40
25000/25000 [==============================] - 6s 231us/step - loss: 0.2351 - acc: 0.9112
Epoch 3/40
25000/25000 [==============================] - 6s 231us/step - loss: 0.1214 - acc: 0.9591
Epoch 4/40
25000/25000 [==============================] - 6s 226us/step - loss: 0.0585 - acc: 0.9816
Epoch 5/40
25000/25000 [==============================] - 6s 229us/step - loss: 0.0313 - acc: 0.9910
Epoch 6/40
25000/25000 [==============================] - 6s 232us/step - loss: 0.0185 - acc: 0.9946
Epoch 7/40
25000/25000 [==============================] - 6s 229us/step - loss: 0.0114 - acc: 0.9969
Epoch 8/40
25000/25000 [==============================] - 6s 235us/step - loss: 0.0120 - acc: 0.9962
Epoch 9/40
25000/25000 [==============================] - 6s 231us/step - loss: 0.0077 - acc: 0.9973
Epoch 10/40
25000/25000 [==============================] - 6s 228us/step - loss: 0.0036 - ac

In [11]:
%%time
# Main evaluation loop: 3s
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

CPU times: user 2.49 s, sys: 231 ms, total: 2.72 s
Wall time: 2.23 s


In [12]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.84032
